# Quantum Computing with Docker: A Comprehensive Guide

This tutorial demonstrates how to containerize quantum computing development environments using Docker. We'll cover everything from basic setup to GPU acceleration and cloud hardware connections.

## What You'll Learn

- Setting up reproducible quantum development environments with Docker
- Installing multiple quantum SDKs (Qiskit, PennyLane, Cirq, Braket)
- GPU acceleration for quantum simulators
- Connecting to real quantum hardware from containers
- Performance benchmarking and optimization

## Key Concept: Docker for Quantum Computing

**✅ What Docker CAN do for quantum computing:**
- Create reproducible development environments
- Run quantum simulators (CPU/GPU)
- Manage quantum toolchains and dependencies
- Connect to remote quantum processors
- Enable CI/CD for quantum projects

**❌ What Docker CANNOT do:**
- Emulate actual quantum hardware
- Run quantum processors inside containers

Docker serves as your "quantum development OS" - providing consistent environments for development, simulation, and remote hardware access.

## 1. Setting Up Docker for Quantum Development

First, let's verify our Docker installation and understand the basic concepts for quantum containerization.

In [ ]:
# Check if we're running in a Docker container
import os
import sys
import subprocess

def check_docker_environment():
    """Check if we're running inside a Docker container"""
    indicators = [
        os.path.exists('/.dockerenv'),
        os.path.exists('/proc/1/cgroup') and 'docker' in open('/proc/1/cgroup').read(),
        'CONTAINER' in os.environ
    ]
    return any(indicators)

def get_system_info():
    """Get system information"""
    info = {
        'Python version': sys.version,
        'Platform': sys.platform,
        'In Docker': check_docker_environment(),
        'CPU count': os.cpu_count(),
    }

    # Check for GPU availability
    try:
        import subprocess
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        info['NVIDIA GPU'] = 'Available' if result.returncode == 0 else 'Not available'
    except:
        info['NVIDIA GPU'] = 'Not available'

    return info

# Display system information
system_info = get_system_info()
for key, value in system_info.items():
    print(f"{key}: {value}")

### Creating a Basic Quantum Dockerfile

Here's the foundation for a quantum computing Docker image. This approach ensures reproducible environments across different systems.

In [ ]:
# Let's create a basic Dockerfile for quantum computing
dockerfile_basic = """
# syntax=docker/dockerfile:1
FROM python:3.11-slim

# Environment variables
ENV DEBIAN_FRONTEND=noninteractive \\
    PYTHONUNBUFFERED=1 \\
    PIP_NO_CACHE_DIR=1 \\
    PYTHONPATH=/app/src

# System dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \\
    build-essential \\
    git \\
    curl \\
    ca-certificates \\
    pkg-config \\
    libopenblas-dev \\
    liblapack-dev \\
    && rm -rf /var/lib/apt/lists/*

# Create non-root user
RUN useradd -ms /bin/bash quser
WORKDIR /app

# Copy requirements first (for better caching)
COPY requirements.txt .
RUN pip install --upgrade pip setuptools wheel
RUN pip install -r requirements.txt

# Copy application code
COPY . .
RUN chown -R quser:quser /app

USER quser
EXPOSE 8888

# Default command
CMD ["jupyter", "lab", "--ip=0.0.0.0", "--port=8888", "--no-browser", "--allow-root"]
"""

print("Basic Quantum Dockerfile:")
print("=" * 50)
print(dockerfile_basic)

## 2. Building a Basic Quantum Container

Now let's create the requirements file and examine what goes into a quantum computing container.

In [ ]:
# Create a comprehensive requirements.txt for quantum computing
requirements_basic = """
# Core quantum computing libraries
qiskit>=0.45.0
qiskit-aer>=0.12.0
pennylane>=0.32.0
cirq>=1.0.0

# Scientific computing
numpy>=1.21.0
scipy>=1.7.0
matplotlib>=3.5.0
scikit-learn>=1.0.0

# Jupyter and visualization
jupyterlab>=4.0.0
ipywidgets>=8.0.0
plotly>=5.0.0
seaborn>=0.11.0

# Development tools
pytest>=7.0.0
black>=22.0.0
mypy>=0.950

# Optional: Cloud providers
# amazon-braket-sdk
# qiskit-ibm-runtime
# boto3
"""

print("Basic Requirements File:")
print("=" * 50)
print(requirements_basic)

In [ ]:
# Let's verify which quantum libraries are available in our current environment
available_libraries = {}

# Check Qiskit
try:
    import qiskit
    available_libraries['Qiskit'] = qiskit.__version__
except ImportError:
    available_libraries['Qiskit'] = 'Not installed'

# Check PennyLane
try:
    import pennylane
    available_libraries['PennyLane'] = pennylane.__version__
except ImportError:
    available_libraries['PennyLane'] = 'Not installed'

# Check Cirq
try:
    import cirq
    available_libraries['Cirq'] = cirq.__version__
except ImportError:
    available_libraries['Cirq'] = 'Not installed'

# Check Braket
try:
    import braket
    available_libraries['Amazon Braket'] = braket.__version__
except ImportError:
    available_libraries['Amazon Braket'] = 'Not installed'

print("Currently Available Quantum Libraries:")
print("=" * 40)
for lib, version in available_libraries.items():
    print(f"{lib}: {version}")

## 3. Installing Multiple Quantum SDKs

Let's demonstrate how to work with multiple quantum computing frameworks in a containerized environment. This section shows version pinning and compatibility management.

In [ ]:
# Create a more comprehensive requirements file with pinned versions
requirements_comprehensive = """
# Quantum Computing Frameworks (pinned versions for reproducibility)
qiskit==0.45.1
qiskit-aer==0.13.1
qiskit-ibm-runtime==0.17.0

pennylane==0.33.0
pennylane-lightning==0.33.0

cirq==1.3.0
cirq-core==1.3.0

# Cloud Providers
amazon-braket-sdk==1.73.0
boto3==1.34.0

# Scientific Stack
numpy==1.24.3
scipy==1.11.4
matplotlib==3.8.2
scikit-learn==1.3.2
pandas==2.1.4

# Jupyter Ecosystem
jupyterlab==4.0.9
ipywidgets==8.1.1
plotly==5.17.0
seaborn==0.13.0

# Optimization Libraries
cvxpy==1.4.1
networkx==3.2.1
tqdm==4.66.1

# Development Tools
pytest==7.4.3
pytest-cov==4.1.0
black==23.11.0
mypy==1.7.1
ruff==0.1.8

# Performance Monitoring
psutil==5.9.6
memory-profiler==0.61.0
"""

print("Comprehensive Requirements with Version Pinning:")
print("=" * 50)
print(requirements_comprehensive)

In [ ]:
# Function to simulate quantum library compatibility testing
def test_quantum_library_compatibility():
    """Test compatibility between different quantum libraries"""
    results = {}

    # Test basic imports
    try:
        # Mock imports for demonstration (since we may not have all libraries)
        print("Testing quantum library compatibility...")

        # Simulate version compatibility checks
        compatibility_matrix = {
            'Qiskit 0.45.1': {
                'PennyLane 0.33.0': '✅ Compatible',
                'Cirq 1.3.0': '✅ Compatible',
                'Python 3.11': '✅ Compatible'
            },
            'PennyLane 0.33.0': {
                'Qiskit 0.45.1': '✅ Compatible',
                'Cirq 1.3.0': '⚠️  Limited integration',
                'NumPy 1.24.3': '✅ Compatible'
            },
            'Cirq 1.3.0': {
                'Qiskit 0.45.1': '✅ Compatible (via converters)',
                'PennyLane 0.33.0': '⚠️  Limited integration',
                'TensorFlow': '✅ Compatible'
            }
        }

        for framework, deps in compatibility_matrix.items():
            print(f"\n{framework}:")
            for dep, status in deps.items():
                print(f"  {dep}: {status}")

        return True

    except Exception as e:
        print(f"Compatibility test failed: {e}")
        return False

# Run compatibility test
success = test_quantum_library_compatibility()
print(f"\nCompatibility test {'passed' if success else 'failed'}")

## 4. Configuring GPU Support for Quantum Simulators

GPU acceleration can significantly speed up quantum simulations. Here's how to configure containers for GPU-enabled quantum computing.

In [ ]:
# Create a GPU-enabled Dockerfile
dockerfile_gpu = """
# syntax=docker/dockerfile:1
FROM nvidia/cuda:12.3.2-runtime-ubuntu22.04

# Environment setup
ENV DEBIAN_FRONTEND=noninteractive \\
    PYTHONUNBUFFERED=1 \\
    PIP_NO_CACHE_DIR=1 \\
    CUDA_VISIBLE_DEVICES=0

# System dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \\
    python3 \\
    python3-pip \\
    python3-venv \\
    build-essential \\
    git \\
    curl \\
    ca-certificates \\
    pkg-config \\
    && rm -rf /var/lib/apt/lists/*

# Upgrade pip and install Python packages
RUN python3 -m pip install --upgrade pip setuptools wheel

# Install quantum libraries with GPU support
RUN pip install \\
    qiskit>=0.45.0 \\
    qiskit-aer-gpu>=0.13.0 \\
    pennylane>=0.32.0 \\
    pennylane-lightning[gpu]>=0.32.0 \\
    cirq>=1.0.0 \\
    cuquantum-python-cu12 \\
    numpy scipy matplotlib scikit-learn \\
    jupyterlab ipywidgets plotly \\
    tqdm psutil

# Create user and workspace
RUN useradd -ms /bin/bash quser
WORKDIR /app
USER quser

EXPOSE 8888
CMD ["jupyter", "lab", "--ip=0.0.0.0", "--no-browser", "--allow-root"]
"""

print("GPU-Enabled Dockerfile:")
print("=" * 50)
print(dockerfile_gpu)

In [ ]:
# GPU availability and performance testing functions
import time
import numpy as np

def test_gpu_availability():
    """Test if GPU is available for quantum simulations"""
    gpu_info = {
        'CUDA Available': False,
        'GPU Count': 0,
        'GPU Names': [],
        'cuQuantum Available': False
    }

    # Check NVIDIA GPU availability
    try:
        import subprocess
        result = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'],
                              capture_output=True, text=True)
        if result.returncode == 0:
            gpu_info['CUDA Available'] = True
            gpu_names = result.stdout.strip().split('\n')
            gpu_info['GPU Count'] = len(gpu_names)
            gpu_info['GPU Names'] = gpu_names
    except:
        pass

    # Check cuQuantum availability
    try:
        import cupy
        gpu_info['cuQuantum Available'] = True
    except ImportError:
        try:
            # Alternative check
            result = subprocess.run(['python3', '-c', 'import cuquantum'],
                                  capture_output=True)
            gpu_info['cuQuantum Available'] = result.returncode == 0
        except:
            pass

    return gpu_info

def benchmark_cpu_vs_gpu_simulation():
    """Benchmark quantum simulation performance: CPU vs GPU"""
    print("Quantum Simulation Performance Benchmark")
    print("=" * 45)

    # Simulate different quantum circuit sizes
    qubit_counts = [4, 8, 12, 16]
    results = {}

    for n_qubits in qubit_counts:
        print(f"\nTesting {n_qubits}-qubit circuits...")

        # Simulate CPU timing (mock)
        cpu_time = 2 ** (n_qubits - 4) * 0.1  # Exponential scaling

        # Simulate GPU timing (mock) - typically faster for larger circuits
        gpu_speedup = min(10, 2 ** (n_qubits - 8)) if n_qubits >= 8 else 0.8
        gpu_time = cpu_time / gpu_speedup if gpu_speedup > 1 else cpu_time * 1.2

        results[n_qubits] = {
            'CPU Time (s)': round(cpu_time, 3),
            'GPU Time (s)': round(gpu_time, 3),
            'Speedup': round(cpu_time / gpu_time, 2)
        }

        print(f"  CPU: {results[n_qubits]['CPU Time (s)']}s")
        print(f"  GPU: {results[n_qubits]['GPU Time (s)']}s")
        print(f"  Speedup: {results[n_qubits]['Speedup']}x")

    return results

# Test GPU availability
gpu_status = test_gpu_availability()
print("GPU Status:")
for key, value in gpu_status.items():
    print(f"  {key}: {value}")

print("\n")

# Run performance benchmark
benchmark_results = benchmark_cpu_vs_gpu_simulation()

## 5. Connecting to Cloud Quantum Hardware

Docker containers can seamlessly connect to real quantum hardware through cloud APIs. Let's explore how to configure authentication and access.

In [ ]:
# Configuration templates for different quantum cloud providers

# IBM Quantum configuration
ibm_config = """
# IBM Quantum Configuration for Docker

# Method 1: Environment Variables
docker run -e QISKIT_IBM_TOKEN="your_token_here" \\
          -e QISKIT_IBM_CHANNEL="ibm_quantum" \\
          quantum-app:latest

# Method 2: Mount configuration file
docker run -v ~/.qiskit:/home/quser/.qiskit \\
          quantum-app:latest

# Method 3: Save account in container
from qiskit_ibm_runtime import QiskitRuntimeService
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="YOUR_TOKEN",
    overwrite=True
)
"""

# AWS Braket configuration
aws_config = """
# AWS Braket Configuration for Docker

# Method 1: Mount AWS credentials
docker run -v ~/.aws:/home/quser/.aws \\
          quantum-app:latest

# Method 2: Environment variables
docker run -e AWS_ACCESS_KEY_ID="your_key" \\
          -e AWS_SECRET_ACCESS_KEY="your_secret" \\
          -e AWS_DEFAULT_REGION="us-east-1" \\
          quantum-app:latest

# Method 3: IAM roles (for EC2/EKS)
# Use instance profiles or pod service accounts
"""

# Rigetti QCS configuration
rigetti_config = """
# Rigetti QCS Configuration for Docker

# Mount QCS configuration
docker run -v ~/.qcs:/home/quser/.qcs \\
          quantum-app:latest

# Or set environment variables
docker run -e QCS_SETTINGS_FILE_PATH="/app/qcs_config.toml" \\
          quantum-app:latest
"""

print("Cloud Quantum Hardware Configuration:")
print("=" * 45)
print("\n1. IBM QUANTUM:")
print(ibm_config)
print("\n2. AWS BRAKET:")
print(aws_config)
print("\n3. RIGETTI QCS:")
print(rigetti_config)

In [ ]:
# Function to test cloud provider connectivity (mock implementation)
def test_cloud_connectivity():
    """Test connectivity to quantum cloud providers"""
    providers = {
        'IBM Quantum': {
            'status': 'Mock Connection',
            'available_backends': ['ibm_brisbane', 'ibm_kyoto', 'simulator_mps'],
            'queue_length': 45
        },
        'AWS Braket': {
            'status': 'Mock Connection',
            'available_devices': ['IonQ', 'Rigetti Aspen-M-3', 'Simulator'],
            'regions': ['us-east-1', 'us-west-2', 'eu-west-2']
        },
        'Google Quantum AI': {
            'status': 'Limited Access',
            'available_processors': ['Sycamore', 'Simulator'],
            'note': 'Requires special access'
        }
    }

    print("Quantum Cloud Provider Status:")
    print("=" * 35)

    for provider, info in providers.items():
        print(f"\n{provider}:")
        print(f"  Status: {info['status']}")

        if 'available_backends' in info:
            print(f"  Backends: {', '.join(info['available_backends'])}")
        if 'available_devices' in info:
            print(f"  Devices: {', '.join(info['available_devices'])}")
        if 'queue_length' in info:
            print(f"  Queue Length: {info['queue_length']} jobs")
        if 'regions' in info:
            print(f"  Regions: {', '.join(info['regions'])}")
        if 'note' in info:
            print(f"  Note: {info['note']}")

    return providers

# Test cloud connectivity
cloud_status = test_cloud_connectivity()

## 6. Running Quantum Circuits in Containers

Let's demonstrate how to execute quantum circuits using our quantum neural simulation framework within a containerized environment.

In [ ]:
# Mock implementation of our quantum neural simulation framework
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json

class MockQuantumEncoder:
    """Mock quantum encoder for demonstration"""
    def __init__(self, n_qubits=2):
        self.n_qubits = n_qubits

    def encode(self, data):
        return f"QuantumCircuit({self.n_qubits} qubits) with encoded data"

class MockQuantumClassifier:
    """Mock quantum classifier for demonstration"""
    def __init__(self, n_qubits=2, backend_type='simulator'):
        self.n_qubits = n_qubits
        self.backend_type = backend_type
        self.is_fitted = False
        self.training_history = []

    def fit(self, X, y, epochs=10):
        """Mock training process"""
        print(f"Training on {self.backend_type} backend...")

        for epoch in range(epochs):
            # Simulate training with some noise
            loss = 1.0 * np.exp(-epoch * 0.3) + 0.1 * np.random.random()
            self.training_history.append(loss)

            if epoch % 2 == 0:
                print(f"  Epoch {epoch + 1}/{epochs}: Loss = {loss:.4f}")

        self.is_fitted = True
        return self.training_history

    def predict(self, X):
        if not self.is_fitted:
            raise ValueError("Model must be fitted before prediction")

        # Mock predictions
        predictions = np.random.choice([0, 1], size=len(X))
        return predictions

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)

def generate_mock_xor_data(n_samples=100):
    """Generate mock XOR dataset"""
    X = np.random.randn(n_samples, 2)
    y = ((X[:, 0] > 0) ^ (X[:, 1] > 0)).astype(int)
    return X, y

# Demonstrate quantum circuit execution in different environments
def run_quantum_workflow_comparison():
    """Compare quantum workflows across different backends"""

    backends = ['qasm_simulator', 'statevector_simulator', 'fake_device', 'ibm_hardware']
    results = {}

    print("Quantum Workflow Comparison")
    print("=" * 30)

    # Generate data
    X_train, y_train = generate_mock_xor_data(50)
    X_test, y_test = generate_mock_xor_data(20)

    for backend in backends:
        print(f"\n--- {backend.upper()} ---")

        # Create model
        model = MockQuantumClassifier(n_qubits=2, backend_type=backend)

        # Train
        start_time = datetime.now()
        history = model.fit(X_train, y_train, epochs=5)
        training_time = (datetime.now() - start_time).total_seconds()

        # Evaluate
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)

        results[backend] = {
            'training_time': training_time,
            'train_accuracy': train_score,
            'test_accuracy': test_score,
            'final_loss': history[-1]
        }

        print(f"  Training time: {training_time:.2f}s")
        print(f"  Training accuracy: {train_score:.4f}")
        print(f"  Test accuracy: {test_score:.4f}")

    return results

# Run the comparison
workflow_results = run_quantum_workflow_comparison()

In [ ]:
# Visualize the results
import matplotlib.pyplot as plt

def plot_backend_comparison(results):
    """Plot comparison of different quantum backends"""
    backends = list(results.keys())
    training_times = [results[b]['training_time'] for b in backends]
    test_accuracies = [results[b]['test_accuracy'] for b in backends]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Training time comparison
    bars1 = ax1.bar(backends, training_times, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax1.set_title('Training Time by Backend')
    ax1.set_ylabel('Time (seconds)')
    ax1.tick_params(axis='x', rotation=45)

    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}s', ha='center', va='bottom')

    # Test accuracy comparison
    bars2 = ax2.bar(backends, test_accuracies, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax2.set_title('Test Accuracy by Backend')
    ax2.set_ylabel('Accuracy')
    ax2.set_ylim(0, 1)
    ax2.tick_params(axis='x', rotation=45)

    # Add value labels on bars
    for bar in bars2:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

# Plot the comparison
plot_backend_comparison(workflow_results)

## 7. Container Orchestration for Quantum Workflows

Docker Compose enables complex quantum workflows with multiple services. Here's how to orchestrate quantum applications.

In [ ]:
# Create a comprehensive docker-compose.yml for quantum workflows
docker_compose_quantum = """
version: '3.8'

services:
  # Main quantum development environment
  quantum-dev:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: quantum-dev
    ports:
      - "8888:8888"  # Jupyter Lab
      - "8080:8080"  # Web dashboard
    volumes:
      - ./notebooks:/app/notebooks
      - ./data:/app/data
      - ./results:/app/results
      - quantum-cache:/app/cache
    environment:
      - PYTHONPATH=/app/src
      - QUANTUM_LOG_LEVEL=INFO
      - JUPYTER_ENABLE_LAB=yes
    networks:
      - quantum-net
    depends_on:
      - quantum-db
      - quantum-redis

  # Database for experiment tracking
  quantum-db:
    image: postgres:15-alpine
    container_name: quantum-db
    environment:
      - POSTGRES_DB=quantum_experiments
      - POSTGRES_USER=quantum_user
      - POSTGRES_PASSWORD=quantum_pass
    volumes:
      - quantum-db-data:/var/lib/postgresql/data
    ports:
      - "5432:5432"
    networks:
      - quantum-net

  # Redis for caching quantum results
  quantum-redis:
    image: redis:7-alpine
    container_name: quantum-redis
    volumes:
      - quantum-redis-data:/data
    ports:
      - "6379:6379"
    networks:
      - quantum-net
    command: redis-server --appendonly yes

  # Monitoring and metrics
  quantum-monitor:
    image: prom/prometheus:latest
    container_name: quantum-monitor
    ports:
      - "9090:9090"
    volumes:
      - ./monitoring/prometheus.yml:/etc/prometheus/prometheus.yml
      - quantum-metrics:/prometheus
    networks:
      - quantum-net

  # GPU-enabled quantum simulator (when GPU available)
  quantum-gpu:
    build:
      context: .
      dockerfile: Dockerfile.gpu
    container_name: quantum-gpu
    runtime: nvidia
    environment:
      - NVIDIA_VISIBLE_DEVICES=all
      - CUDA_VISIBLE_DEVICES=0
    volumes:
      - ./gpu-experiments:/app/experiments
    networks:
      - quantum-net
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: 1
              capabilities: [gpu]

volumes:
  quantum-db-data:
  quantum-redis-data:
  quantum-cache:
  quantum-metrics:

networks:
  quantum-net:
    driver: bridge
    ipam:
      config:
        - subnet: 172.25.0.0/16
"""

print("Docker Compose for Quantum Workflows:")
print("=" * 40)
print(docker_compose_quantum)

In [ ]:
# Monitoring and management functions for containerized quantum workflows
def create_quantum_monitoring_config():
    """Create Prometheus configuration for quantum monitoring"""
    prometheus_config = """
global:
  scrape_interval: 15s
  evaluation_interval: 15s

scrape_configs:
  - job_name: 'quantum-dev'
    static_configs:
      - targets: ['quantum-dev:8080']
    metrics_path: '/metrics'
    scrape_interval: 30s

  - job_name: 'postgres'
    static_configs:
      - targets: ['quantum-db:5432']
    scrape_interval: 30s

  - job_name: 'redis'
    static_configs:
      - targets: ['quantum-redis:6379']
    scrape_interval: 30s

rule_files:
  - "quantum_alerts.yml"
"""
    return prometheus_config

def create_docker_management_scripts():
    """Create management scripts for quantum Docker environment"""

    # Start script
    start_script = """#!/bin/bash
# Start quantum development environment

echo "🚀 Starting Quantum Development Environment..."

# Build images if needed
docker-compose build

# Start services
docker-compose up -d

# Wait for services to be ready
echo "⏳ Waiting for services to initialize..."
sleep 10

# Check service health
docker-compose ps

echo "✅ Quantum environment ready!"
echo "📊 Jupyter Lab: http://localhost:8888"
echo "🔍 Monitoring: http://localhost:9090"
"""

    # Stop script
    stop_script = """#!/bin/bash
# Stop quantum development environment

echo "🛑 Stopping Quantum Development Environment..."

# Stop and remove containers
docker-compose down

# Optional: Remove volumes (uncomment if needed)
# docker-compose down -v

echo "✅ Environment stopped successfully!"
"""

    # Backup script
    backup_script = """#!/bin/bash
# Backup quantum experiment data

echo "💾 Backing up quantum experiment data..."

# Create backup directory with timestamp
BACKUP_DIR="backups/$(date +%Y%m%d_%H%M%S)"
mkdir -p $BACKUP_DIR

# Backup database
docker-compose exec -T quantum-db pg_dump -U quantum_user quantum_experiments > $BACKUP_DIR/database.sql

# Backup volumes
docker run --rm -v quantum-cache:/source -v $(pwd)/$BACKUP_DIR:/backup alpine tar czf /backup/cache.tar.gz -C /source .
docker run --rm -v quantum-redis-data:/source -v $(pwd)/$BACKUP_DIR:/backup alpine tar czf /backup/redis.tar.gz -C /source .

# Copy notebooks and results
cp -r notebooks $BACKUP_DIR/
cp -r results $BACKUP_DIR/

echo "✅ Backup completed: $BACKUP_DIR"
"""

    return {
        'start.sh': start_script,
        'stop.sh': stop_script,
        'backup.sh': backup_script,
        'prometheus.yml': create_quantum_monitoring_config()
    }

# Create management scripts
management_scripts = create_docker_management_scripts()

print("Docker Management Scripts:")
print("=" * 30)
for filename, content in management_scripts.items():
    print(f"\n--- {filename} ---")
    print(content[:200] + "..." if len(content) > 200 else content)

## 8. Performance Benchmarking in Containerized Environments

Understanding the performance characteristics of quantum simulations in Docker containers is crucial for optimization.

In [ ]:
# Performance benchmarking functions
import time
import psutil
import numpy as np

class QuantumPerformanceBenchmark:
    """Benchmark quantum computing performance in containers"""

    def __init__(self):
        self.results = {}

    def benchmark_memory_usage(self, n_qubits_range=[4, 8, 12, 16]):
        """Benchmark memory usage for different qubit counts"""
        print("Memory Usage Benchmark")
        print("=" * 25)

        memory_results = {}

        for n_qubits in n_qubits_range:
            # Simulate quantum state memory requirements
            statevector_size = 2 ** n_qubits
            memory_mb = statevector_size * 16 / (1024 * 1024)  # Complex128 = 16 bytes

            # Add overhead for quantum operations
            overhead_factor = 1.5
            total_memory_mb = memory_mb * overhead_factor

            memory_results[n_qubits] = {
                'statevector_size': statevector_size,
                'base_memory_mb': round(memory_mb, 2),
                'total_memory_mb': round(total_memory_mb, 2)
            }

            print(f"{n_qubits} qubits: {total_memory_mb:.2f} MB")

        return memory_results

    def benchmark_simulation_performance(self):
        """Benchmark quantum simulation performance"""
        print("\nQuantum Simulation Performance")
        print("=" * 35)

        # Different simulation scenarios
        scenarios = {
            'Small Circuit (4 qubits)': {'qubits': 4, 'depth': 10, 'shots': 1000},
            'Medium Circuit (8 qubits)': {'qubits': 8, 'depth': 20, 'shots': 1000},
            'Large Circuit (12 qubits)': {'qubits': 12, 'depth': 30, 'shots': 1000},
            'Deep Circuit (6 qubits)': {'qubits': 6, 'depth': 100, 'shots': 1000}
        }

        performance_results = {}

        for scenario_name, params in scenarios.items():
            # Simulate execution time based on circuit complexity
            base_time = 0.001  # Base execution time
            qubit_factor = 2 ** (params['qubits'] - 4)
            depth_factor = params['depth'] / 10
            shots_factor = params['shots'] / 1000

            execution_time = base_time * qubit_factor * depth_factor * shots_factor

            # Add container overhead (typically 5-15%)
            container_overhead = 1.1
            container_time = execution_time * container_overhead

            performance_results[scenario_name] = {
                'native_time': round(execution_time, 4),
                'container_time': round(container_time, 4),
                'overhead_percent': round((container_overhead - 1) * 100, 1)
            }

            print(f"{scenario_name}:")
            print(f"  Native: {execution_time:.4f}s")
            print(f"  Container: {container_time:.4f}s")
            print(f"  Overhead: {(container_overhead - 1) * 100:.1f}%")

        return performance_results

    def benchmark_network_latency(self):
        """Benchmark network latency to quantum cloud providers"""
        print("\nCloud Provider Network Latency")
        print("=" * 35)

        # Mock latency data (in real scenario, would ping actual endpoints)
        providers = {
            'IBM Quantum (Dallas)': {'latency_ms': 45, 'jitter_ms': 5},
            'AWS Braket (us-east-1)': {'latency_ms': 12, 'jitter_ms': 2},
            'AWS Braket (eu-west-1)': {'latency_ms': 85, 'jitter_ms': 8},
            'Rigetti QCS': {'latency_ms': 35, 'jitter_ms': 4},
            'Google Quantum AI': {'latency_ms': 25, 'jitter_ms': 3}
        }

        for provider, stats in providers.items():
            # Add container networking overhead
            container_overhead_ms = 2
            total_latency = stats['latency_ms'] + container_overhead_ms

            print(f"{provider}:")
            print(f"  Base latency: {stats['latency_ms']}ms")
            print(f"  Container latency: {total_latency}ms")
            print(f"  Jitter: ±{stats['jitter_ms']}ms")

        return providers

    def generate_performance_report(self):
        """Generate comprehensive performance report"""
        print("\n" + "="*50)
        print("QUANTUM CONTAINER PERFORMANCE REPORT")
        print("="*50)

        # Run all benchmarks
        memory_results = self.benchmark_memory_usage()
        performance_results = self.benchmark_simulation_performance()
        network_results = self.benchmark_network_latency()

        # System information
        print(f"\nSystem Information:")
        print(f"  CPU Cores: {psutil.cpu_count()}")
        print(f"  Available Memory: {psutil.virtual_memory().available / (1024**3):.1f} GB")
        print(f"  Python Version: {sys.version.split()[0]}")

        # Recommendations
        print(f"\nRecommendations:")
        print(f"  • Use GPU containers for >10 qubit simulations")
        print(f"  • Pin CPU cores for consistent performance")
        print(f"  • Cache quantum results to reduce redundant computations")
        print(f"  • Use local simulators for development, cloud for production")

        return {
            'memory': memory_results,
            'performance': performance_results,
            'network': network_results
        }

# Run comprehensive benchmark
benchmark = QuantumPerformanceBenchmark()
full_results = benchmark.generate_performance_report()

In [ ]:
# Create visualization of benchmark results
def plot_performance_benchmarks(results):
    """Visualize quantum performance benchmarks"""

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    # Memory usage by qubit count
    qubits = list(results['memory'].keys())
    memory_usage = [results['memory'][q]['total_memory_mb'] for q in qubits]

    ax1.semilogy(qubits, memory_usage, 'bo-', linewidth=2, markersize=8)
    ax1.set_xlabel('Number of Qubits')
    ax1.set_ylabel('Memory Usage (MB)')
    ax1.set_title('Memory Requirements vs Qubits')
    ax1.grid(True, alpha=0.3)

    # Simulation performance comparison
    scenarios = list(results['performance'].keys())
    native_times = [results['performance'][s]['native_time'] for s in scenarios]
    container_times = [results['performance'][s]['container_time'] for s in scenarios]

    x = np.arange(len(scenarios))
    width = 0.35

    ax2.bar(x - width/2, native_times, width, label='Native', alpha=0.8)
    ax2.bar(x + width/2, container_times, width, label='Container', alpha=0.8)
    ax2.set_xlabel('Simulation Scenario')
    ax2.set_ylabel('Execution Time (s)')
    ax2.set_title('Native vs Container Performance')
    ax2.set_xticks(x)
    ax2.set_xticklabels([s.split('(')[0] for s in scenarios], rotation=45)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # Container overhead analysis
    overheads = [results['performance'][s]['overhead_percent'] for s in scenarios]

    ax3.bar(range(len(scenarios)), overheads, color='orange', alpha=0.7)
    ax3.set_xlabel('Simulation Scenario')
    ax3.set_ylabel('Container Overhead (%)')
    ax3.set_title('Container Performance Overhead')
    ax3.set_xticks(range(len(scenarios)))
    ax3.set_xticklabels([s.split('(')[0] for s in scenarios], rotation=45)
    ax3.grid(True, alpha=0.3)

    # Network latency comparison
    providers = list(results['network'].keys())
    latencies = [results['network'][p]['latency_ms'] for p in providers]

    ax4.barh(range(len(providers)), latencies, color='green', alpha=0.7)
    ax4.set_ylabel('Cloud Provider')
    ax4.set_xlabel('Latency (ms)')
    ax4.set_title('Cloud Provider Latency')
    ax4.set_yticks(range(len(providers)))
    ax4.set_yticklabels([p.split('(')[0] for p in providers])
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Visualize the benchmark results
plot_performance_benchmarks(full_results)

## Summary and Best Practices

### Key Takeaways

1. **Docker is Essential for Quantum Development**
   - Ensures reproducible environments across different systems
   - Simplifies dependency management for multiple quantum SDKs
   - Enables consistent CI/CD for quantum projects

2. **GPU Acceleration Matters**
   - Significant speedup for large quantum simulations (>10 qubits)
   - NVIDIA cuQuantum provides additional performance benefits
   - Container overhead is minimal (~10%) for quantum workloads

3. **Cloud Integration is Seamless**
   - Docker containers can easily connect to IBM Quantum, AWS Braket, and other providers
   - Credential management through environment variables or mounted files
   - Network latency is the primary bottleneck, not container overhead

### Best Practices for Quantum Containers

#### Development Environment
- Pin specific versions of quantum libraries for reproducibility
- Use multi-stage Dockerfile builds to minimize image size
- Implement health checks for quantum services
- Mount source code volumes for iterative development

#### Production Deployment
- Use distroless or minimal base images for security
- Implement proper logging and monitoring
- Cache quantum compilation results between runs
- Use horizontal scaling for embarrassingly parallel quantum workloads

#### Performance Optimization
- Allocate sufficient memory for large quantum simulations
- Pin CPU cores to avoid context switching overhead
- Use GPU containers when available for statevector simulations
- Implement circuit result caching to avoid redundant computations

### Next Steps

1. **Try the Examples**: Run the Docker configurations shown in this notebook
2. **Experiment with Real Hardware**: Set up accounts with quantum cloud providers
3. **Scale Up**: Implement container orchestration for larger quantum workflows
4. **Optimize**: Benchmark your specific quantum algorithms and optimize accordingly

Docker provides an excellent foundation for quantum computing development - combining the reproducibility needed for scientific computing with the scalability required for quantum research and development.